# Import & Preprocessing (for raw TF-IDF system/model)

In [1]:
# import
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# load dataset
df = pd.read_csv('scraping/halodocnew_1.csv')
print(df.shape)

# check dataset
df.head()

(9979, 5)


,url,title,title_count,content,content_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...",10,"Bagi orang tua yang memiliki kendaraan, memili...",540
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",9,Sebagai makanan pokok beras harus disimpan den...,564
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",9,Merokok tidak hanya berbahaya untuk dirimu sen...,649
3,https://www.halodoc.com/sering-terkena-polusi-...,"Sering Terkena Polusi, Ketahui Berbagi Cara B...",8,"Sehari-hari, manusia terpapar polusi dalam jum...",523
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,Ini Risiko dan Manfaat Implan Payudara yang P...,9,Implan payudara adalah prosedur bedah kosmetik...,547


In [2]:
# remove 'content' that contains 'halodoc' (dianggap iklan/promosi, tidak terlalu berpengaruh ke konteks dokumen/teks)
df['content'] = df['content'].apply(lambda x: '.'.join([s for s in x.split('.') if 'halodoc' not in s.lower()]))

In [3]:
# make information retrieval system
# Query = df['title']
# Document = df['content']

# Preprocessing
# 1. Case Folding
# 2. Tokenizing
# 3. Filtering
# 4. Stemming

# Case Folding
df['title'] = df['title'].str.lower()
df['content'] = df['content'].str.lower()
df.head()


,url,title,title_count,content,content_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",10,"bagi orang tua yang memiliki kendaraan, memili...",540
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",9,sebagai makanan pokok beras harus disimpan den...,564
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",9,merokok tidak hanya berbahaya untuk dirimu sen...,649
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",8,"sehari-hari, manusia terpapar polusi dalam jum...",523
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,547


In [4]:
nltk.download('punkt')

# Tokenize the title column
df['title_tokens'] = df['title'].apply(nltk.word_tokenize)
df['title_count'] = df['title_tokens'].apply(len)

# Tokenize the content column
df['content_tokens'] = df['content'].apply(nltk.word_tokenize)
df['content_count'] = df['content_tokens'].apply(len)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Magezter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df.head()

,url,title,title_count,content,content_count,title_tokens,content_tokens
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"[ibu, ,, perhatikan, 5, hal, ini, saat, memili...","[bagi, orang, tua, yang, memiliki, kendaraan, ..."
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"[catat, ,, ini, 6, tips, menyimpan, beras, aga...","[sebagai, makanan, pokok, beras, harus, disimp..."
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"[waspada, ,, ini, 6, bahaya, asap, rokok, bagi...","[merokok, tidak, hanya, berbahaya, untuk, diri..."
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"[sering, terkena, polusi, ,, ketahui, berbagi,...","[sehari-hari, ,, manusia, terpapar, polusi, da..."
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"[ini, risiko, dan, manfaat, implan, payudara, ...","[implan, payudara, adalah, prosedur, bedah, ko..."


In [6]:
# Filtering
# Remove stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

# Remove stopwords from title
df['title_tokens_stopwords'] = df['title_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['title_tokens_stopwords_count'] = df['title_tokens_stopwords'].apply(len)

# Remove stopwords from content
df['content_tokens_stopwords'] = df['content_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['content_tokens_stopwords_count'] = df['content_tokens_stopwords'].apply(len)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Magezter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,title,title_count,content,content_count,title_tokens,content_tokens,title_tokens_stopwords,title_tokens_stopwords_count,content_tokens_stopwords,content_tokens_stopwords_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"[ibu, ,, perhatikan, 5, hal, ini, saat, memili...","[bagi, orang, tua, yang, memiliki, kendaraan, ...","[,, perhatikan, 5, memilih, car, seat, newborn]",7,"[orang, tua, memiliki, kendaraan, ,, memilihca...",339
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"[catat, ,, ini, 6, tips, menyimpan, beras, aga...","[sebagai, makanan, pokok, beras, harus, disimp...","[catat, ,, 6, tips, menyimpan, beras, berkutu]",7,"[makanan, pokok, beras, disimpan, kualitasnya,...",347
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"[waspada, ,, ini, 6, bahaya, asap, rokok, bagi...","[merokok, tidak, hanya, berbahaya, untuk, diri...","[waspada, ,, 6, bahaya, asap, rokok, kesehatan...",8,"[merokok, berbahaya, dirimu, orang-orang, seki...",438
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"[sering, terkena, polusi, ,, ketahui, berbagi,...","[sehari-hari, ,, manusia, terpapar, polusi, da...","[terkena, polusi, ,, ketahui, berbagi, bersihk...",7,"[sehari-hari, ,, manusia, terpapar, polusi, as...",342
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"[ini, risiko, dan, manfaat, implan, payudara, ...","[implan, payudara, adalah, prosedur, bedah, ko...","[risiko, manfaat, implan, payudara]",4,"[implan, payudara, prosedur, bedah, kosmetik, ...",362


In [7]:
# Stemming (using Sastrawi)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemming title
df['title_tokens_stopwords_stemmed'] = df['title_tokens_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df['title_tokens_stopwords_stemmed_count'] = df['title_tokens_stopwords_stemmed'].apply(len)

# Stemming content
df['content_tokens_stopwords_stemmed'] = df['content_tokens_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df['content_tokens_stopwords_stemmed_count'] = df['content_tokens_stopwords_stemmed'].apply(len)
df.head()

,url,title,title_count,content,content_count,title_tokens,content_tokens,title_tokens_stopwords,title_tokens_stopwords_count,content_tokens_stopwords,content_tokens_stopwords_count,title_tokens_stopwords_stemmed,title_tokens_stopwords_stemmed_count,content_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"[ibu, ,, perhatikan, 5, hal, ini, saat, memili...","[bagi, orang, tua, yang, memiliki, kendaraan, ...","[,, perhatikan, 5, memilih, car, seat, newborn]",7,"[orang, tua, memiliki, kendaraan, ,, memilihca...",339,"[, perhati, 5, pilih, car, seat, newborn]",7,"[orang, tua, milik, kendara, , memilihcar, sea...",339
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"[catat, ,, ini, 6, tips, menyimpan, beras, aga...","[sebagai, makanan, pokok, beras, harus, disimp...","[catat, ,, 6, tips, menyimpan, beras, berkutu]",7,"[makanan, pokok, beras, disimpan, kualitasnya,...",347,"[catat, , 6, tips, simpan, beras, kutu]",7,"[makan, pokok, beras, simpan, kualitas, turun,...",347
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"[waspada, ,, ini, 6, bahaya, asap, rokok, bagi...","[merokok, tidak, hanya, berbahaya, untuk, diri...","[waspada, ,, 6, bahaya, asap, rokok, kesehatan...",8,"[merokok, berbahaya, dirimu, orang-orang, seki...",438,"[waspada, , 6, bahaya, asap, rokok, sehat, anak]",8,"[rokok, bahaya, diri, orang, sekitar, , bayi, ...",438
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"[sering, terkena, polusi, ,, ketahui, berbagi,...","[sehari-hari, ,, manusia, terpapar, polusi, da...","[terkena, polusi, ,, ketahui, berbagi, bersihk...",7,"[sehari-hari, ,, manusia, terpapar, polusi, as...",342,"[kena, polusi, , tahu, bagi, sih, paru]",7,"[hari, , manusia, papar, polusi, asap, kendara...",342
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"[ini, risiko, dan, manfaat, implan, payudara, ...","[implan, payudara, adalah, prosedur, bedah, ko...","[risiko, manfaat, implan, payudara]",4,"[implan, payudara, prosedur, bedah, kosmetik, ...",362,"[risiko, manfaat, implan, payudara]",4,"[implan, payudara, prosedur, bedah, kosmetik, ...",362


- Stopwords removal takes 0.8s
- Stemming takes 108m 14.7s

In [8]:
# # save df (for later use)
# df.to_csv('tf-idf_halodocnew_1.csv', index=False)

# New Dataset

In [9]:
# add new dataset ()
# import
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import json

def read_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data


file = 'train.json'
data = read_json(file)
df_json = pd.DataFrame(data)
df_json

,id,text,label
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
1,1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,Merdeka.com - Indonesia Corruption Watch ( ICW...,Merdeka.com - Indonesia Corruption Watch ( ICW...
2,1503039338-pembagian-sepeda-usai-upacara-penur...,Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,"Usai prosesi penurunan bendera dilakukan , Jok..."
3,1501747636-korut-dan-as-saling-pamer-rudal-mem...,"Jakarta , CNN Indonesia - - Militer Amerika Se...","Jakarta , CNN Indonesia - - Militer Amerika Se..."
4,1475197245-indonesia-kejar-target-jadi-destina...,"Jakarta , CNN Indonesia - - Wisata halal menja...","Jakarta , CNN Indonesia - - Wisata halal menja..."
...,...,...,...
14078,1479257116-hebat-anak-penjual-nasi-uduk-diangk...,JANGAN engkau terus meratapi nasib hanya karen...,Filosofi inilah yang menjadi pegangan bagi ana...
14079,1519720680-dua-desainer-indonesia-di-balik-bus...,"Jakarta , CNN Indonesia - - Aktris Lindsay L...","Jakarta , CNN Indonesia - - Aktris Lindsay L..."
14080,1515704400-doktor-ui-temukan-deteksi-kanker-me...,Jakarta ( ANTARA News ) - Doktor Biomedik dari...,Jakarta ( ANTARA News ) - Doktor Biomedik dari...
14081,1519724700-peneliti-indonesia-dan-australia-ke...,"Jakarta , CNN Indonesia - - Tim peneliti Indon...","Jakarta , CNN Indonesia - - Tim peneliti Indon..."


In [10]:
df_json['text'][14078]

'JANGAN engkau terus meratapi nasib hanya karena takdir terlahir dari keluarga tak mampu . Tapi yakinlah , nasibmu akan berubah , jika engkau pasrah dalam iman kepada Allah dalam segala perbuatanmu . Filosofi inilah yang menjadi pegangan bagi anak ke delapan dari sepuluh bersaudara ini . Lahir dari anak pasangan dimana ayahnya berprofesi sebagai pekerja serabutan dan ibunya seorang penjual nasi uduk dan pisang goreng keliling , tidak membuat semangat lelaki ini minder . Justru sebaliknya , ia memiliki tekad dan semangat , merubah derajat hidup keluarganya . Perjuangannya untuk menjadikan hidupnya bisa berguna bagi orang lain dari kesederhanaannya , membawanya menjadi seorang polisi . Pria dengan semangat dan cita-cita mulia ini adalah Wawan Wahyudin . Ia lahir di kota kembang , Bandung tanggal 7 September 1963 dari pasangan Akhil dan Siti Rohmah . “ Saya tak ingin meratapi nasib , hanya karena terlahir dari orangtua yang penuh kesederhanaan . Alhamdulillah Allah menyempurnakan cita-cit

In [11]:
df_json['text_word_count'] = df_json['text'].apply(lambda x: len(x.split()))
df_json['label_word_count'] = df_json['label'].apply(lambda x: len(x.split()))

In [12]:
print(df_json['text_word_count'].mean())
print(df_json['label_word_count'].mean())

346.77781722644323
74.24305900731379


In [13]:
# import
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [14]:
# Hapus kata-kata yang berada sebelum dash '-' pertama
def remove_words_before_dash(text):
    sentences = text.split('. ')  # Split the text into sentences
    first_sentence = sentences[0]  # Get the first sentence

    if '-' in first_sentence:
        pattern = r'^.*?-\s*'
        match = re.sub(pattern, '', first_sentence)
        match = match.lstrip('-')
        sentences[0] = match

    return '. '.join(sentences)  # Reconstruct the modified text


# Hapus kata-kata yang berada sebelum dash '-' pertama tiap kalimat pertama pada baris df_json['text']
df_json['text'] = df_json['text'].apply(lambda x: remove_words_before_dash(x))
df_json

,id,text,label,text_word_count,label_word_count
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58
1,1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,Indonesia Corruption Watch ( ICW ) meminta Kom...,Merdeka.com - Indonesia Corruption Watch ( ICW...,214,85
2,1503039338-pembagian-sepeda-usai-upacara-penur...,Presiden Joko Widodo ( Jokowi ) memimpin upaca...,"Usai prosesi penurunan bendera dilakukan , Jok...",289,67
3,1501747636-korut-dan-as-saling-pamer-rudal-mem...,Militer Amerika Serikat akan melakukan uji co...,"Jakarta , CNN Indonesia - - Militer Amerika Se...",249,66
4,1475197245-indonesia-kejar-target-jadi-destina...,Wisata halal menjadi salah satu fokus pengemb...,"Jakarta , CNN Indonesia - - Wisata halal menja...",374,85
...,...,...,...,...,...
14078,1479257116-hebat-anak-penjual-nasi-uduk-diangk...,JANGAN engkau terus meratapi nasib hanya karen...,Filosofi inilah yang menjadi pegangan bagi ana...,696,89
14079,1519720680-dua-desainer-indonesia-di-balik-bus...,Aktris Lindsay Lohan mencuri perhatian publ...,"Jakarta , CNN Indonesia - - Aktris Lindsay L...",338,69
14080,1515704400-doktor-ui-temukan-deteksi-kanker-me...,Doktor Biomedik dari Fakultas Kedokteran Unive...,Jakarta ( ANTARA News ) - Doktor Biomedik dari...,509,98
14081,1519724700-peneliti-indonesia-dan-australia-ke...,Tim peneliti Indonesia yang bergabung dalam...,"Jakarta , CNN Indonesia - - Tim peneliti Indon...",421,83


In [15]:
df_json['text_lower'] = df_json['text'].str.lower()
df_json['label_lower'] = df_json['label'].str.lower()
df_json.head(1)

,id,text,label,text_word_count,label_word_count,text_lower,label_lower
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...


In [16]:
# Tokenize the text and label column
df_json['text_tokens'] = df_json['text_lower'].apply(nltk.word_tokenize)
df_json['label_tokens'] = df_json['label_lower'].apply(nltk.word_tokenize)
df_json.head(1)

,id,text,label,text_word_count,label_word_count,text_lower,label_lower,text_tokens,label_tokens
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...,"[dokter, ryan, thamrin, ,, yang, terkenal, lew...","[dokter, lula, kamal, yang, merupakan, selebri..."


In [17]:
# Remove stopwords from text and label column
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

df_json['text_tokens_stopwords'] = df_json['text_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df_json['label_tokens_stopwords'] = df_json['label_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df_json.head(1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Magezter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,text,label,text_word_count,label_word_count,text_lower,label_lower,text_tokens,label_tokens,text_tokens_stopwords,label_tokens_stopwords
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...,"[dokter, ryan, thamrin, ,, yang, terkenal, lew...","[dokter, lula, kamal, yang, merupakan, selebri...","[dokter, ryan, thamrin, ,, terkenal, acara, do...","[dokter, lula, kamal, selebriti, rekan, kerja,..."


In [18]:
# Stemming (using Sastrawi)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df_json['text_tokens_stopwords_stemmed'] = df_json['text_tokens_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df_json['label_tokens_stopwords_stemmed'] = df_json['label_tokens_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df_json.head(1)

,id,text,label,text_word_count,label_word_count,text_lower,label_lower,text_tokens,label_tokens,text_tokens_stopwords,label_tokens_stopwords,text_tokens_stopwords_stemmed,label_tokens_stopwords_stemmed
0,1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"Dokter Ryan Thamrin , yang terkenal lewat aca...",Dokter Lula Kamal yang merupakan selebriti sek...,330,58,"dokter ryan thamrin , yang terkenal lewat aca...",dokter lula kamal yang merupakan selebriti sek...,"[dokter, ryan, thamrin, ,, yang, terkenal, lew...","[dokter, lula, kamal, yang, merupakan, selebri...","[dokter, ryan, thamrin, ,, terkenal, acara, do...","[dokter, lula, kamal, selebriti, rekan, kerja,...","[dokter, ryan, thamrin, , kenal, acara, dokter...","[dokter, lula, kamal, selebriti, rekan, kerja,..."


- Stopwords removal takes 1.4s
- Stemming takes 99m 39.4s

In [19]:
# # save df_json (for later use) 
# df_json.to_csv('tf-idf_train.csv', index=False)